# Agent Long-Term Memory

Long-term memory persists user facts across sessions using stores with semantic search.

**What you'll learn:**
- Store persists data across sessions and threads
- PostgresStore with embeddings enables semantic search
- Tools access store via runtime.store
- Context provides user identification
- Namespaces organize memories hierarchically
- Semantic search finds relevant memories by meaning

## Memory Comparison

| Type | Storage | Use Case | Persistence |
|------|---------|----------|-------------|
| **Short-term** | Checkpointer | Conversation history | Session |
| **Long-term** | Store | User preferences, facts | Cross-session |

In [ ]:
import sys
sys.path.append('../')

import os
from dotenv import load_dotenv
load_dotenv()

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain.agents import create_agent
from langchain.messages import HumanMessage
from langchain.tools import tool, ToolRuntime
from pydantic import BaseModel

from scripts import base_tools

In [ ]:
model = ChatGoogleGenerativeAI(model='gemini-2.5-flash')

system_prompt = """You are a helpful assistant with long-term memory.

MEMORY TOOLS USAGE:

save_user_memory - Save when user shares NEW information:
- Food preferences (diet, likes, dislikes, allergies)
- Work information (role, company, interests)  
- Hobbies and personal interests
- Location and timezone preferences

get_user_memory - Retrieve specific category:
- When answering questions about past preferences
- When user asks "what do you know about me?"
- When making personalized recommendations

web_search - Retrieve Real-time information
- When user ask about news or latest update
- When user ask about real-time information

GUIDELINES:
- Always save when user shares personal information
- Use stored preferences to personalize responses
- Be conversational and natural"""

## Setup PostgresStore

In [ ]:
from dataclasses import dataclass
from langgraph.store.postgres import PostgresStore
from langgraph.checkpoint.postgres import PostgresSaver
import psycopg

embeddings = GoogleGenerativeAIEmbeddings(model='gemini-embedding-001')

def embed(texts: list[str]):
    return embeddings.embed_documents(texts)

pg_conn = psycopg.connect(os.getenv("POSTGRESQL_URL"), autocommit=True, prepare_threshold=0)
store = PostgresStore(pg_conn, index={"embed": embed, "dims": 768})
store.setup()

# context is immutable
@dataclass
class UserContext:
    user_id: str


## Memory Tools with ToolRuntime

In [ ]:
@tool
def save_user_memory(category: str, information: dict, runtime: ToolRuntime):
    """Save user preference or information to long-term memory.
    
    Args:
        category: Category of information (e.g., 'food', 'work', 'hobbies', 'location')
        information: Dictionary containing the information to save
        
    Examples:
        category='food', information={'diet': 'vegetarian', 'likes': ['pasta', 'pizza']}
        category='work', information={'role': 'Data Scientist', 'interests': ['AI', 'ML']}
    """
    store = runtime.store
    user_id = runtime.context.user_id
    namespace = (user_id, "preferences")
    
    store.put(namespace, category, information)
    return f"Saved {category} preferences for {user_id}"

@tool
def get_user_memory(category: str, runtime: ToolRuntime):
    """Retrieve user preference or information from long-term memory.
    
    Args:
        category: Category of information to retrieve (e.g., 'food', 'work', 'hobbies')
        
    Returns:
        Stored information for the category or "not found" message
    """
    store = runtime.store
    user_id = runtime.context.user_id
    namespace = (user_id, "preferences")
    
    item = store.get(namespace, category)
    if item:
        return f"{category}: {item.value}"
    return f"No '{category}' information found"

## Agent with Long-Term Memory

In [ ]:
pg_saver = PostgresSaver(pg_conn)
pg_saver.setup()

agent = create_agent(
    model=model,
    tools=[save_user_memory, get_user_memory, base_tools.web_search],
    checkpointer=pg_saver,
    store=store,
    context_schema=UserContext,
    system_prompt=system_prompt
)

## Save User Preferences

In [ ]:
# Save user information
config = {"configurable": {"thread_id": "session_1"}}
user_context = UserContext(user_id='kgptalkie')

response = agent.invoke({
    "messages": [HumanMessage("I'm a vegetarian and I love pasta, pizza, and Italian food")]
}, config=config, context=user_context)


response['messages'][-1].content

In [ ]:
# Save work information
response = agent.invoke({
    "messages": [HumanMessage("I work as a Data Scientist and I'm interested in AI and machine learning")]
}, config=config, context=user_context)

response['messages'][-1].content

## Retrieve in New Session

In [ ]:
# New thread - different conversation, but same user
new_config = {"configurable": {"thread_id": "session_2"}}

response = agent.invoke({
    "messages": [HumanMessage("What do you know about my food preferences?")]
}, config=new_config, context=user_context)

response['messages'][-1].content

In [ ]:
# Ask for recommendations based on stored preferences
response = agent.invoke({
    "messages": [HumanMessage("Can you recommend a restaurant for me? use web search")]
}, config=new_config, context=user_context)

response['messages'][-1].content

## Semantic Search

In [ ]:
# Direct store access for semantic search
namespace = ("user_123", "preferences")
memories = store.search(namespace, query="What does the user like to eat?", limit=3)

for m in memories:
    print(f"{m.key}: {m.value}")